In [13]:
import ssl
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import re
import statistics
import matplotlib.pyplot as py
import pandas as pd

In [ ]:
#To Do: get features and make data frame

In [9]:
def get_links(pages = 1):
    #gets the links from trulia properties page 
    # if length of list is zero, captcha blocked request so use a new VPN location 
    #pages: number of pages of trula to scrape
    #returns: list of links
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    i = 0
    property_links = []
    while i < pages:
        url = 'https://www.trulia.com/NY/New_York/' + "%s_p/"%str(i)
        req = Request(url, headers={"User-Agent": 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage , 'html.parser')
        for link in soup.findAll('a'):
            l = str(link.get('href'))
            if sum(map(str.isdigit, l[-17:])) == 15: property_links.append("https://www.trulia.com" + l)
        property_links = property_links[::2]
        print("finishing loop" + " " + str(i+1))
        i += 1
        time.sleep(5)
        
    return(property_links)

In [4]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [26]:
feature_dict_list = {}
all_features = []
all_links = []
for index,url in enumerate(get_links()): #iterates through all URLs from the get_links func
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()  
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    feature_dict = {}

    for sou in soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"}):
        clean_data = str(sou).split("gmLKqq\">")
        clean_data = clean_data[1].split("</span>")[0]
        dict_info = clean_data.split(':')
        if len(dict_info)>1:
            feature_dict[dict_info[0]] = dict_info[1]


    for bed_html in soup.findAll('span', attrs={'data-testid':"home-summary-size-bedrooms"}):
        number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
        if number_beds:
            feature_dict["beds"] = int(number_beds[0].split(" ")[0])

    for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
        number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
        if number_baths:
            feature_dict["baths"] = int(number_baths[0].split(" ")[0])

    for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
        floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
        property_info["floor_space"] = {}
        if floor_footage:
            feature_dict["floor_space"]["value"] = int(floor_footage[0].split(" ")[0].replace(",", ""))

    all_features.append(feature_dict);all_links.append(url)
    df = pd.DataFrame.from_dict(all_features)
    


finishing loop 1


In [25]:
df

NameError: name 'baths' is not defined

In [ ]:
for house in all_features:
    for f in house:
        dict_key,dict_value = f.split(':')

In [62]:
# For ignoring SSL certificate errors  (reused code)

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [63]:
#url = input("Enter URL: ")
url = 'https://www.trulia.com/p/ny/staten-island/365-victory-blvd-staten-island-ny-10301--2008726710'

In [28]:
# Use Mozilla access agent

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

In [29]:
# Creating a BeautifulSoup object of the html page for easy extraction of data.

soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
property_info = {}

In [30]:
# This code block will get you a one liner description of the listed property

for meta in soup.findAll('meta', attrs={'name': 'description'}):
    try:
        property_info['description'] = meta['content']
        break
    except: 
        pass

property_info['description']

KeyError: 'description'

In [31]:
#Store url 

property_info['link'] = url

property_info['link']

'https://www.trulia.com/p/ny/new-york/309-e-houston-st-1a-new-york-ny-10002--2478352572'

In [33]:
#Get beds, baths, floorspace

for bed_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bedrooms"}):
    number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
    if number_beds:
        property_info["beds"] = int(number_beds[0].split(" ")[0])

for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
    number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
    if number_baths:
        property_info["baths"] = int(number_baths[0].split(" ")[0])

for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
    floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
    property_info["floor_space"] = {}
    if floor_footage:
        property_info["floor_space"]["value"] = int(floor_footage[0].split(" ")[0].replace(",", ""))
        property_info["floor_space"]["unit"] = floor_footage[0].split(" ")[1]

property_info


{'link': 'https://www.trulia.com/p/ny/new-york/309-e-houston-st-1a-new-york-ny-10002--2478352572'}

In [69]:
#Extracts features for given house
for index, sou in enumerate(soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"})):
    clean_data = str(sou).split("gmLKqq\">")
    clean_data = clean_data[1].split("</span>")[0]
clean_data

'Lot Area: 3001 sqft'